In [49]:

from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
import numpy as np
import sklearn 
from sklearn import preprocessing
from random import seed
from random import randrange
from math import sqrt
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import train_test_split 
from sklearn.metrics import r2_score
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from subprocess import check_output
from datetime import time
%matplotlib inline
from scipy import sparse
from sklearn.decomposition import PCA
from sklearn.cross_validation import ShuffleSplit, train_test_split
from sklearn import metrics
from sklearn.learning_curve import learning_curve
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.grid_search import GridSearchCV
import pandas as pd
from pandas.tools.plotting import scatter_matrix
import urllib
import requests
import seaborn

In [61]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

In [62]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['time']
del df['datenew']
#SETTING DATE COLUMN AS INDEX
df = df.set_index('date')

In [5]:
df.head(10)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,...,18,11,0,66000,0,0,0,0,0,0


In [63]:
def mae(org,pred):
    return mean_absolute_error(org,pred)
def rmse(org,pred):
    return np.sqrt(mean_squared_error(org,pred))
def r2s(org,pred):
    return r2_score(org,pred)
def mape(org,pred):
    y_true, y_pred = np.array(org), np.array(pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


In [64]:
def scores(model,yorg,ypred):
    print("The scores for the model : ",model)
    print("MAE    :    ",mae(yorg,ypred))
    print("RMSE   :    ",rmse(yorg,ypred))
    print("R2     :    ",r2s(yorg,ypred))
    print("MAPE   :    ",mape(yorg,ypred))

Model without Scaling

In [65]:
data1 = df.drop('Appliances', 1)
X = data1
y = df.Appliances
 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=1)

gbrt=GradientBoostingRegressor(n_estimators=100) 


In [66]:
print("X Split size")
print(len(X_train) , len(X_test))
print("Y Split size")
print(len(y_train) , len(y_test))

X Split size
13814 5921
Y Split size
13814 5921


In [67]:
gbrt.fit(X_train, y_train) 



GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [68]:
ypredtrain=gbrt.predict(X_train)
ypredtest=gbrt.predict(X_test)


Scores without scaling

In [69]:
print("Scores for test")
scores("GBM",y_test,ypredtest)

Scores for test
The scores for the model :  GBM
MAE    :     45.72461860215228
RMSE   :     85.54414582514036
R2     :     0.3074664787710045
MAPE   :     48.94709970689996


In [70]:
print("Scores for train")
scores("GBM",y_train,ypredtrain)

Scores for train
The scores for the model :  GBM
MAE    :     43.39040409505426
RMSE   :     80.66444346110487
R2     :     0.3794780718034906
MAPE   :     47.900407183832385


Modeling with scaling

In [34]:
data1 = df.drop('Appliances', 1)
X = data1
y = df.Appliances.values
y = y.reshape(len(y),-1)

In [35]:
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [39]:
y=y.ravel()

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=1)

gbrt=GradientBoostingRegressor(n_estimators=100) 


In [41]:
print("X Split size")
print(len(X_train) , len(X_test))
print("Y Split size")
print(len(y_train) , len(y_test))

X Split size
13814 5921
Y Split size
13814 5921


In [42]:
gbrt.fit(X_train, y_train) 

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [46]:
ypredtrain1=gbrt.predict(X_train)
ypredtest1=gbrt.predict(X_test)

Scores with scaling

In [53]:
print("Scores for test")
scores("GBM",y_test,ypredtest1)

Scores for test
The scores for the model :  GBM
MAE    :     0.04269151342132954
RMSE   :     0.07992813651477858
R2     :     0.3078070968965061
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


In [54]:
print("Scores for train")
scores("GBM",y_train,ypredtrain1)

Scores for train
The scores for the model :  GBM
MAE    :     0.04055177952808811
RMSE   :     0.07538733033748118
R2     :     0.3794780718034907
MAPE   :     inf


C:\Users\chethan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


array([0.03429399, 0.07398755, 0.09710842, ..., 0.17581236, 0.12756492,
       0.11794979])